In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 25 19:16:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers

In [3]:
HF_TOKEN = "hugging-face-token"

In [4]:
import os

In [5]:
os.environ["HF_TOKEN"] = HF_TOKEN

In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
model_name = "google/gemma-3-1b-it"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [11]:
tokenizer("Hello, World!")

{'input_ids': [2, 9259, 236764, 4109, 236888], 'attention_mask': [1, 1, 1, 1, 1]}

In [12]:
input_conversation = [
    { "role": "user", "content": "Which is the best place to learn GenAI?" },
    { "role": "assistant", "content": "The best place to learn AI is" }
]

In [13]:
input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=True,
)
input_tokens

[2,
 105,
 2364,
 107,
 24249,
 563,
 506,
 1791,
 1977,
 531,
 3449,
 8471,
 12553,
 236881,
 106,
 107,
 105,
 4368,
 107,
 818,
 1791,
 1977,
 531,
 3449,
 12498,
 563,
 106,
 107]

In [15]:
input_detokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False,
    continue_final_message=True,
)
input_detokens

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI is'

In [17]:
output_label = " GenAI Cohort 1.0 by ChaiCode and Piyush Garg use Coupon PIYUSH10 Please"
full_conversation = input_detokens + output_label + tokenizer.eos_token
full_conversation

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI is GenAI Cohort 1.0 by ChaiCode and Piyush Garg use Coupon PIYUSH10 Please<eos>'

In [18]:
input_tokenized = tokenizer(full_conversation, return_tensors="pt", add_special_tokens=False).to(device)["input_ids"]
input_tokenized

tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,   1977,
            531,   3449,   8471,  12553, 236881,    106,    107,    105,   4368,
            107,    818,   1791,   1977,    531,   3449,  12498,    563,   8471,
          12553, 105657,    632, 236743, 236770, 236761, 236771,    684, 119806,
           4809,    532, 168222,   1974, 102629,   1161,  97887,   6108, 236874,
          52907, 236770, 236771,   7323,      1]], device='cuda:0')

In [21]:
input_ids = input_tokenized[:, :-1].to(device)
target_ids = input_tokenized[:, 1:].to(device)
print(f"input_ids: {input_ids}")
print(f"target_ids: {target_ids}")

detokenized_input = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
print(f"detokenized_input: {detokenized_input}")

input_ids: tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,   1977,
            531,   3449,   8471,  12553, 236881,    106,    107,    105,   4368,
            107,    818,   1791,   1977,    531,   3449,  12498,    563,   8471,
          12553, 105657,    632, 236743, 236770, 236761, 236771,    684, 119806,
           4809,    532, 168222,   1974, 102629,   1161,  97887,   6108, 236874,
          52907, 236770, 236771,   7323]], device='cuda:0')
target_ids: tensor([[   105,   2364,    107,  24249,    563,    506,   1791,   1977,    531,
           3449,   8471,  12553, 236881,    106,    107,    105,   4368,    107,
            818,   1791,   1977,    531,   3449,  12498,    563,   8471,  12553,
         105657,    632, 236743, 236770, 236761, 236771,    684, 119806,   4809,
            532, 168222,   1974, 102629,   1161,  97887,   6108, 236874,  52907,
         236770, 236771,   7323,      1]], device='cuda:0')
detokenized_input: ['user\nWhich is the best pl

In [22]:
import torch.nn as nn
def calculate_loss(logits, labels):
    loss_fn = nn.CrossEntropyLoss(reduction="none")
    cross_entropy = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
    return cross_entropy

In [23]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
).to(device)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [24]:
from torch.optim import AdamW
model.train()

optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

for _ in range(10):
  out = model(input_ids=input_ids)
  loss = calculate_loss(out.logits, target_ids).mean()
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(loss.item())

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


11.6875
5.125
3.046875
2.03125
1.0390625
0.67578125
0.51953125
0.427734375
0.392578125
0.365234375


In [25]:
input_prompt = [
    { "role": "user", "content": "Which is the best place to learn GenAI?" }
]

input = tokenizer.apply_chat_template(
    conversation=input_prompt,
    return_tensors="pt",
    tokenize=True,
).to(device)

output = model.generate(input, max_new_tokens=35)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

['user\nWhich is the best place to learn GenAI?\nmodel\nThe best place to learn AI is GenAI Cohort 1.0 by ChaiCode and Piyush Garg use Coupon PIYUSH10 Please']
